In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import sklearn.model_selection as skm
from sklearn.metrics import confusion_matrix

In [2]:
df_train = pd.read_csv('train.csv')

In [3]:
df_train.shape

(8693, 14)

In [4]:
df_train.isna().sum()


PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [5]:
# cols = list(df_train.columns)[1:7]
# print(cols)

In [6]:
# idxNa = []

# idxNa, _ = np.where(df_train.isna())
# idxNa = list(set(idxNa))

# idxs = {}
# for i in cols:
#     idxs[i] = df_train[df_train[i].isna()].index
    
# naIdx = {}
# for col in cols:
#     naIdx[col] = [True if idxNa[i] in (idxs[col]) else False for i in range(len(idxNa))]

In [7]:
# train - inGroup

group = [i[0:4] for i in df_train['PassengerId']]
inGroup = [1 if group.count(i) > 1 else 0 for i in group]

df_train['PassengerId_inGroup'] = inGroup

In [8]:
# train

values = {"RoomService": 0, "FoodCourt": 0, 
          "ShoppingMall": 0, "Spa": 0, "VRDeck": 0, "VIP": False}

df_train = df_train.fillna(value=values)

In [9]:
# train - totalFee

totalFee = df_train['RoomService']+ df_train['FoodCourt']+ df_train['ShoppingMall']+df_train['Spa']+df_train['VRDeck']
df_train['totalFee']=totalFee

# if totalFee==0 and CryoSleep isna, then set CryoSleep to True, else False

In [10]:
ageEuropa = np.mean(df_train['Age'][df_train['HomePlanet']=='Europa'])
ageEarth = np.mean(df_train['Age'][df_train['HomePlanet']=='Earth'])
ageMars = np.mean(df_train['Age'][df_train['HomePlanet']=='Mars'])

In [11]:
# train - destination

idxTRAPByFee = df_train[df_train['totalFee']>=1600][df_train['Destination'].isna()].index
idxPSOByFee = df_train[df_train['totalFee']<1600][df_train['totalFee']>1000][df_train['Destination'].isna()].index
idx55ByFee = df_train[df_train['totalFee']<1000][df_train['Destination'].isna()].index
df_train.loc[idxTRAPByFee,['Destination']] = 'TRAPPIST-1e'
df_train.loc[idxPSOByFee,['Destination']] = 'PSO J318.5-22'
df_train.loc[idx55ByFee,['Destination']] = '55 Cancri e'

C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\3361102049.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxTRAPByFee = df_train[df_train['totalFee']>=1600][df_train['Destination'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\3361102049.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxPSOByFee = df_train[df_train['totalFee']<1600][df_train['totalFee']>1000][df_train['Destination'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\3361102049.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idx55ByFee = df_train[df_train['totalFee']<1000][df_train['Destination'].isna()].index


In [12]:
# train - cryosleep

idxCS = df_train[df_train['totalFee']==0][df_train['CryoSleep'].isna()].index
df_train.loc[idxCS,['CryoSleep']] = True
idxNCS = df_train[df_train['totalFee']!=0][df_train['CryoSleep'].isna()].index
df_train.loc[idxNCS,['CryoSleep']] = False

C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\3202240518.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxCS = df_train[df_train['totalFee']==0][df_train['CryoSleep'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\3202240518.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxNCS = df_train[df_train['totalFee']!=0][df_train['CryoSleep'].isna()].index


In [13]:
# train - Cabin

df_train[['deck','num','side']] = df_train.Cabin.str.split("/",expand=True)
df_train = df_train.drop(columns=['Name', 'PassengerId', 'Cabin', 'num'])

In [14]:
# from visualization, passengers on decks A, B and C are all from Europa
# passengers on deck G are all from Earth
# passengers on deck F are from either Earth or Mars
# passengers on deck D are most likely from Mars, then Europa
# passengers on deck E are most likely from Earth, then Mars, then Europa

# train - HomePlant

idxDeckABC = df_train[df_train['deck'].isin(['A','B','C','T'])][df_train['HomePlanet'].isna()].index
df_train.loc[idxDeckABC,['HomePlanet']] = 'Europa'
idxDeckEFG = df_train[df_train['deck'].isin(['E','F','G'])][df_train['HomePlanet'].isna()].index
df_train.loc[idxDeckEFG,['HomePlanet']] = 'Earth'
idxDeckEFG = df_train[df_train['deck']=='D'][df_train['HomePlanet'].isna()].index
df_train.loc[idxDeckEFG,['HomePlanet']] = 'Mars'

# idxDeckABC = df_train[df_train['deck'].isin(['A','B','C','T'])][df_train['HomePlanet'].isna()].index
# df_train.loc[idxDeckABC,['HomePlanet']] = 'Europa'
# idxDeckEFG = df_train[df_train['deck']=='G'][df_train['HomePlanet'].isna()].index # take out E,F
# df_train.loc[idxDeckEFG,['HomePlanet']] = 'Earth'
# # idxDeckEFG = df_train[df_train['deck']=='D'][df_train['HomePlanet'].isna()].index
# # df_train.loc[idxDeckEFG,['HomePlanet']] = 'Mars'

C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\2244768207.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxDeckABC = df_train[df_train['deck'].isin(['A','B','C','T'])][df_train['HomePlanet'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\2244768207.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxDeckEFG = df_train[df_train['deck'].isin(['E','F','G'])][df_train['HomePlanet'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\2244768207.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxDeckEFG = df_train[df_train['deck']=='D'][df_train['HomePlanet'].isna()].index


In [15]:
# if totalFee > 2000, then it is likely the Homeplanet is Europa
# if totalFee < 2000 & room service > 340, then it is likely the HomePlanet is Mars
# if totalFee < 2000 & room service < 340, then it is likely the HomePlanet is Earth

# train - HomePlanet

idxEuropaByFee = df_train[df_train['totalFee']>=2000][df_train['HomePlanet'].isna()].index
idxMarsByFee = df_train[df_train['totalFee']<2000][df_train['RoomService']>=340][df_train['HomePlanet'].isna()].index
idxEarthByFee = df_train[df_train['totalFee']<2000][df_train['RoomService']<340][df_train['HomePlanet'].isna()].index
df_train.loc[idxEuropaByFee,['HomePlanet']] = 'Europa'
df_train.loc[idxMarsByFee,['HomePlanet']] = 'Mars'
df_train.loc[idxEarthByFee,['HomePlanet']] = 'Earth'

C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\332782212.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxEuropaByFee = df_train[df_train['totalFee']>=2000][df_train['HomePlanet'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\332782212.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxMarsByFee = df_train[df_train['totalFee']<2000][df_train['RoomService']>=340][df_train['HomePlanet'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\332782212.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxEarthByFee = df_train[df_train['totalFee']<2000][df_train['RoomService']<340][df_train['HomePlanet'].isna()].index


In [16]:
# train - Age

idxAgeEarth = df_train[df_train['HomePlanet']=='Earth'][df_train['Age'].isna()].index
idxAgeEuropa = df_train[df_train['HomePlanet']=='Europa'][df_train['Age'].isna()].index
idxAgeMars = df_train[df_train['HomePlanet']=='Mars'][df_train['Age'].isna()].index
df_train.loc[idxAgeEarth,['Age']] = int(np.floor(ageEarth))
df_train.loc[idxAgeEuropa,['Age']] = int(np.floor(ageEuropa))
df_train.loc[idxAgeMars,['Age']] = int(np.floor(ageMars))

C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\3708677144.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxAgeEarth = df_train[df_train['HomePlanet']=='Earth'][df_train['Age'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\3708677144.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxAgeEuropa = df_train[df_train['HomePlanet']=='Europa'][df_train['Age'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\3708677144.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxAgeMars = df_train[df_train['HomePlanet']=='Mars'][df_train['Age'].isna()].index


In [17]:
# train - deck

idxEarth = df_train[df_train['HomePlanet']=='Earth'][df_train['deck'].isna()].index
df_train.loc[idxEarth,['deck']] = 'G'
idxEuropa = df_train[df_train['HomePlanet']=='Europa'][df_train['deck'].isna()].index
df_train.loc[idxEuropa,['deck']] = 'B'
idxMars = df_train[df_train['HomePlanet']=='Mars'][df_train['deck'].isna()].index
df_train.loc[idxMars,['deck']] = 'F'

# idxEarthBelow26 = df_train[df_train['HomePlanet']=='Earth'][df_train['Age']<=26][df_train['deck'].isna()].index
# df_train.loc[idxEarthBelow26,['deck']] = 'G'
# idxEarthAbove26 = df_train[df_train['HomePlanet']=='Earth'][df_train['Age']>26][df_train['deck'].isna()].index
# df_train.loc[idxEarthAbove26,['deck']] = 'F'
# idxEuropaAbove500 = df_train[df_train['HomePlanet']=='Europa'][df_train['VRDeck']>500][df_train['deck'].isna()].index
# df_train.loc[idxEuropaAbove500,['deck']] = 'B' # could be A,B or C
# idxEuropaBelow500 = df_train[df_train['HomePlanet']=='Europa'][df_train['VRDeck']<=500][df_train['deck'].isna()].index
# df_train.loc[idxEuropaBelow500,['deck']] = 'E' # could be D or E
# idxMarsAbove30 = df_train[df_train['HomePlanet']=='Mars'][df_train['Age']>=30][df_train['deck'].isna()].index
# df_train.loc[idxMarsAbove30,['deck']] = 'D'
# idxMarsBelow30 = df_train[df_train['HomePlanet']=='Mars'][df_train['Age']<30][df_train['deck'].isna()].index
# df_train.loc[idxMarsBelow30,['deck']] = 'F'

C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\3936988800.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxEarth = df_train[df_train['HomePlanet']=='Earth'][df_train['deck'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\3936988800.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxEuropa = df_train[df_train['HomePlanet']=='Europa'][df_train['deck'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\3936988800.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxMars = df_train[df_train['HomePlanet']=='Mars'][df_train['deck'].isna()].index


In [18]:
# train - side

idxSideP = df_train[df_train['deck'].isin(['A','B','C','E','G'])][df_train['side'].isna()].index
df_train.loc[idxSideP,['side']] = 'P'
idxSideS = df_train[df_train['deck'].isin(['D','F','T'])][df_train['side'].isna()].index
df_train.loc[idxSideS,['side']] = 'S'

C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\3273352895.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxSideP = df_train[df_train['deck'].isin(['A','B','C','E','G'])][df_train['side'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\3273352895.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxSideS = df_train[df_train['deck'].isin(['D','F','T'])][df_train['side'].isna()].index


In [19]:
# train

df_train['isPort'] = [1 if i=='P' else 0 for i in df_train['side']]
df_train['isVIP'] = [1 if i=='True' else 0 for i in df_train['VIP']]
df_train['isCryoSleep'] = [1 if i=='True' else 0 for i in df_train['CryoSleep']]
df_train['fromEuropa'] = [1 if i=='Europa' else 0 for i in df_train['HomePlanet']]
df_train['to55'] = [1 if i=='55 Cancri e' else 0 for i in df_train['Destination']]
df_train['SPA200'] = [1 if i>=200 else 0 for i in df_train['Spa']]

In [20]:
# train

X=df_train.drop(columns=['side', 'VIP', 'CryoSleep'])
# df_train_drop = df_train_drop.dropna()
# df_train_drop = df_train_drop.reset_index()
# df_train_drop = df_train_drop.drop(columns=['index'])

In [21]:
# remove insignificant columns
# from df_train_drop changed to X
# X = df_train_drop.drop(columns=['FoodCourt', 'ShoppingMall', 'Spa'])

In [22]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(np.array(X['Destination']).reshape(-1, 1))
X[['Destination_0','Destination_1','Destination_2']] = enc.transform(np.array(X['Destination']).reshape(-1, 1)).toarray()

In [23]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(np.array(X['HomePlanet']).reshape(-1, 1))
X[['HomePlanet_0','HomePlanet_1','HomePlanet_2']] = enc.transform(np.array(X['HomePlanet']).reshape(-1, 1)).toarray()

In [24]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(np.array(X['deck']).reshape(-1, 1))
X[['deck_0','deck_1','deck_2','deck_3','deck_4','deck_5','deck_6','deck_7']] = enc.transform(np.array(X['deck']).reshape(-1, 1)).toarray()

In [25]:
# X_valid = X.loc[idxNa]
# y_valid = X_valid['Transported']
# X_valid = X_valid.reset_index()
# X_valid = X_valid.drop(columns=['Transported', 'index', 'Destination', 'HomePlanet', 'deck'])
# # print(X_valid)

# X_train = X.drop(idxNa)
# y_train = X_train['Transported']
# X_train = X_train.reset_index()
# X_train = X_train.drop(columns=['Transported', 'index', 'Destination', 'HomePlanet', 'deck'])
# # print(X_train)


In [26]:
y = X['Transported']
X = X.drop(columns=['Transported'])
X = X.drop(columns=['Destination', 'HomePlanet', 'deck'])


In [27]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state=2)

#-----------------------------------------CV--------------------------------------------
# kfold = skm.KFold(10,
#               random_state=1,
#               shuffle=True)
# # xgb params booster='gbtree', alpha=6, eta=0.1, gamma=0, lambda=1, learning_rate=0.01, n_estimators=1000, reg_alpha=0, reg_lambda=10,subsample=0.8
# parameters = {'n_estimators':[1000,900,1200]}
# rf_cv = XGBClassifier(booster='gbtree', alpha=6, gamma=0,colsample_bytree=0.5, 
#                       learning_rate=0.01, reg_alpha=0, n_estimators=900,
#                       reg_lambda=5, subsample=0.8, max_depth=6)

# # rfc params
# # parameters = {'n_estimators':list(np.arange(140,160,4)),
# #              'ccp_alpha':[0.1,0.01]}
# # rf_cv = RandomForestClassifier(random_state=1, criterion='entropy', max_features=4)

# # adaboost params
# # parameters = {'learning_rate':[0.1, 0.2, 0.5, 1], 
# #               'n_estimators':list(np.arange(50,200,50))}

# grid = skm.GridSearchCV(rf_cv,
#                         parameters,
#                         refit = True,
#                         cv=kfold,
#                         scoring='accuracy')
# grid.fit(X_train, y_train)
# # RFC.fit(X, y)
# RFC = grid.best_estimator_
#-----------------------------------------CV--------------------------------------------
# RFC = XGBClassifier(booster='gbtree', alpha=6, gamma=0,colsample_bytree=0.5, 
#                       learning_rate=0.01, reg_alpha=0, n_estimators=900,
#                       reg_lambda=5, subsample=0.8, max_depth=6)
# RFC.fit(X,y)
# RFC.fit(X_train, y_train)
# pred_valid = RFC.predict(X_valid).tolist()
# pred_train = RFC.predict(X_train).tolist()

# y_train = y_train.tolist()
# y_valid = y_valid.tolist()
# print(RFC)

In [28]:
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)
# # # RFC = AdaBoostClassifier(learning_rate=1, n_estimators=160, random_state=1)
# # RFC = RandomForestClassifier(max_features=4, n_estimators=144, criterion='gini',ccp_alpha=0.0008)
# RFC = XGBClassifier()
# RFC.fit(X_train, y_train)

# pred_valid = RFC.predict(X_valid).tolist()
# pred_train = RFC.predict(X_train).tolist()

# y_train = y_train.tolist()
# y_valid = y_valid.tolist()

In [29]:
# cnt_train = 0
# for i in range(len(y_train)):
#     if y_train[i]==pred_train[i]:
#         cnt_train += 1
# cnt_valid = 0      
# for i in range(len(y_valid)):
#     if y_valid[i]==pred_valid[i]:
#         cnt_valid += 1

# print('Accuracy of training data set: ' + str(cnt_train / len(y_train)))
# print('Accuracy of validation data set: ' + str(cnt_valid / len(y_valid)))

# # Accuracy of training data set: 0.8360655737704918
# # Accuracy of validation data set: 0.8131109833237493 -- use all features and tuned params
# # Accuracy of training data set: 0.9115616911130284
# # Accuracy of validation data set: 0.8033352501437608 -- use all features, no tuned params

In [30]:
# confusion_matrix(y_train, pred_train)

In [31]:
# confusion_matrix(y_valid, pred_valid)

In [32]:
# accuracy = [y_valid[i]==pred_valid[i] for i in range(len(y_valid))]
# check = pd.DataFrame()
# check['idx'] = list(idxNa)
# check['y_valid'] = list(y_valid)
# check['pred_valid'] = list(pred_valid)
# check['acc'] = accuracy
# for col in cols:
#     check[col] = naIdx[col]
# check.to_csv('check_acc.csv',index=False)

In [33]:
# RFC = AdaBoostClassifier(learning_rate=1, n_estimators=100, random_state=1)
# RFC = RandomForestClassifier(max_features=4, n_estimators=144, criterion='entropy',ccp_alpha=0.0005)
RFC = XGBClassifier()
RFC = XGBClassifier(booster='gbtree', alpha=6, gamma=0,colsample_bytree=0.5, 
                      learning_rate=0.01, reg_alpha=0, n_estimators=900,
                      reg_lambda=5, subsample=0.8, max_depth=6)

RFC.fit(X, y)
pred = RFC.predict(X).tolist()
pred = [True if i==1 else False for i in pred]
y = y.tolist()
cnt = 0
for i in range(len(y)):
    if y[i]==pred[i]:
        cnt += 1
        
print('Accuracy of full data set: ' + str(cnt / len(y)))
print(confusion_matrix(y, pred))

Accuracy of full data set: 0.8345795467617624
[[3491  824]
 [ 614 3764]]


### test dataset

In [34]:
df_test = pd.read_csv('test.csv')

In [35]:
# test 0 inGroup
psgid = df_test['PassengerId']
group = [i[0:4] for i in df_test['PassengerId']]
inGroup = [1 if group.count(i) > 1 else 0 for i in group]

df_test['PassengerId_inGroup'] = inGroup

In [36]:
# test

values = {"RoomService": 0, "FoodCourt": 0, 
          "ShoppingMall": 0, "Spa": 0, "VRDeck": 0, "VIP": False}

df_test = df_test.fillna(value=values)

In [37]:
# test - totalFee

totalFee = df_test['RoomService']+ df_test['FoodCourt']+ df_test['ShoppingMall']+df_test['Spa']+df_test['VRDeck']
df_test['totalFee']=totalFee

# if totalFee==0 and CryoSleep isna, then set CryoSleep to True, else False

In [38]:
# test - destination

idxTRAPByFee = df_test[df_test['totalFee']>=1600][df_test['Destination'].isna()].index
idxPSOByFee = df_test[df_test['totalFee']<1600][df_test['totalFee']>1000][df_test['Destination'].isna()].index
idx55ByFee = df_test[df_test['totalFee']<1000][df_test['Destination'].isna()].index
df_test.loc[idxTRAPByFee,['Destination']] = 'TRAPPIST-1e'
df_test.loc[idxPSOByFee,['Destination']] = 'PSO J318.5-22'
df_test.loc[idx55ByFee,['Destination']] = '55 Cancri e'

C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\1661410121.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxTRAPByFee = df_test[df_test['totalFee']>=1600][df_test['Destination'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\1661410121.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxPSOByFee = df_test[df_test['totalFee']<1600][df_test['totalFee']>1000][df_test['Destination'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\1661410121.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idx55ByFee = df_test[df_test['totalFee']<1000][df_test['Destination'].isna()].index


In [39]:
# test - cryosleep

idxCS = df_test[df_test['totalFee']==0][df_test['CryoSleep'].isna()].index
df_test.loc[idxCS,['CryoSleep']] = True
idxNCS = df_test[df_test['totalFee']!=0][df_test['CryoSleep'].isna()].index
df_test.loc[idxNCS,['CryoSleep']] = False

C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\307260986.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxCS = df_test[df_test['totalFee']==0][df_test['CryoSleep'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\307260986.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxNCS = df_test[df_test['totalFee']!=0][df_test['CryoSleep'].isna()].index


In [40]:
# test - Cabin

df_test[['deck','num','side']] = df_test.Cabin.str.split("/",expand=True)
df_test = df_test.drop(columns=['Name', 'PassengerId', 'Cabin', 'num'])

In [41]:
# test - HomePlanet

idxDeckABC = df_test[df_test['deck'].isin(['A','B','C','T'])][df_test['HomePlanet'].isna()].index
df_test.loc[idxDeckABC,['HomePlanet']] = 'Europa'
idxDeckEFG = df_test[df_test['deck'].isin(['E','F','G'])][df_test['HomePlanet'].isna()].index
df_test.loc[idxDeckEFG,['HomePlanet']] = 'Earth'
idxDeckEFG = df_test[df_test['deck']=='D'][df_test['HomePlanet'].isna()].index
df_test.loc[idxDeckEFG,['HomePlanet']] = 'Mars'

# idxDeckABC = df_test[df_test['deck'].isin(['A','B','C','T'])][df_test['HomePlanet'].isna()].index
# df_test.loc[idxDeckABC,['HomePlanet']] = 'Europa'
# idxDeckEFG = df_test[df_test['deck']=='G'][df_test['HomePlanet'].isna()].index # take out E,F
# df_test.loc[idxDeckEFG,['HomePlanet']] = 'Earth'
# # idxDeckEFG = df_train[df_train['deck']=='D'][df_train['HomePlanet'].isna()].index
# # df_train.loc[idxDeckEFG,['HomePlanet']] = 'Mars'

C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\2542191232.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxDeckABC = df_test[df_test['deck'].isin(['A','B','C','T'])][df_test['HomePlanet'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\2542191232.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxDeckEFG = df_test[df_test['deck'].isin(['E','F','G'])][df_test['HomePlanet'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\2542191232.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxDeckEFG = df_test[df_test['deck']=='D'][df_test['HomePlanet'].isna()].index


In [42]:
# test - HomePlanet

idxEuropaByFee = df_test[df_test['totalFee']>=2000][df_test['HomePlanet'].isna()].index
idxMarsByFee = df_test[df_test['totalFee']<2000][df_test['RoomService']>=340][df_test['HomePlanet'].isna()].index
idxEarthByFee = df_test[df_test['totalFee']<2000][df_test['RoomService']<340][df_test['HomePlanet'].isna()].index
df_test.loc[idxEuropaByFee,['HomePlanet']] = 'Europa'
df_test.loc[idxMarsByFee,['HomePlanet']] = 'Mars'
df_test.loc[idxEarthByFee,['HomePlanet']] = 'Earth'

C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\1704111286.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxEuropaByFee = df_test[df_test['totalFee']>=2000][df_test['HomePlanet'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\1704111286.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxMarsByFee = df_test[df_test['totalFee']<2000][df_test['RoomService']>=340][df_test['HomePlanet'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\1704111286.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxEarthByFee = df_test[df_test['totalFee']<2000][df_test['RoomService']<340][df_test['HomePlanet'].isna()].index


In [43]:
# test - Age

idxAgeEarth = df_test[df_test['HomePlanet']=='Earth'][df_test['Age'].isna()].index
idxAgeEuropa = df_test[df_test['HomePlanet']=='Europa'][df_test['Age'].isna()].index
idxAgeMars = df_test[df_test['HomePlanet']=='Mars'][df_test['Age'].isna()].index
df_test.loc[idxAgeEarth,['Age']] = int(np.floor(ageEarth))
df_test.loc[idxAgeEuropa,['Age']] = int(np.floor(ageEuropa))
df_test.loc[idxAgeMars,['Age']] = int(np.floor(ageMars))

C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\4194361286.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxAgeEarth = df_test[df_test['HomePlanet']=='Earth'][df_test['Age'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\4194361286.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxAgeEuropa = df_test[df_test['HomePlanet']=='Europa'][df_test['Age'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\4194361286.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxAgeMars = df_test[df_test['HomePlanet']=='Mars'][df_test['Age'].isna()].index


In [44]:
# test - deck

idxEarth = df_test[df_test['HomePlanet']=='Earth'][df_test['deck'].isna()].index
df_test.loc[idxEarth,['deck']] = 'G'
idxEuropa = df_test[df_test['HomePlanet']=='Europa'][df_test['deck'].isna()].index
df_test.loc[idxEuropa,['deck']] = 'B'
idxMars = df_test[df_test['HomePlanet']=='Mars'][df_test['deck'].isna()].index
df_test.loc[idxMars,['deck']] = 'F'

# idxEarthBelow26 = df_test[df_test['HomePlanet']=='Earth'][df_test['Age']<=26][df_test['deck'].isna()].index
# df_test.loc[idxEarthBelow26,['deck']] = 'G'
# idxEarthAbove26 = df_test[df_test['HomePlanet']=='Earth'][df_test['Age']>26][df_test['deck'].isna()].index
# df_test.loc[idxEarthAbove26,['deck']] = 'F'
# idxEuropaAbove500 = df_test[df_test['HomePlanet']=='Europa'][df_test['VRDeck']>500][df_test['deck'].isna()].index
# df_test.loc[idxEuropaAbove500,['deck']] = 'B' # could be A,B or C
# idxEuropaBelow500 = df_test[df_test['HomePlanet']=='Europa'][df_test['VRDeck']<=500][df_test['deck'].isna()].index
# df_test.loc[idxEuropaBelow500,['deck']] = 'E' # could be D or E
# idxMarsAbove30 = df_test[df_test['HomePlanet']=='Mars'][df_test['Age']>=30][df_test['deck'].isna()].index
# df_test.loc[idxMarsAbove30,['deck']] = 'D'
# idxMarsBelow30 = df_test[df_test['HomePlanet']=='Mars'][df_test['Age']<30][df_test['deck'].isna()].index
# df_test.loc[idxMarsBelow30,['deck']] = 'F'

C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\1243641460.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxEarth = df_test[df_test['HomePlanet']=='Earth'][df_test['deck'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\1243641460.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxEuropa = df_test[df_test['HomePlanet']=='Europa'][df_test['deck'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\1243641460.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxMars = df_test[df_test['HomePlanet']=='Mars'][df_test['deck'].isna()].index


In [45]:
# test - side

idxSideP = df_test[df_test['deck'].isin(['A','B','C','E','G'])][df_test['side'].isna()].index
df_test.loc[idxSideP,['side']] = 'P'
idxSideS = df_test[df_test['deck'].isin(['D','F','T'])][df_test['side'].isna()].index
df_test.loc[idxSideS,['side']] = 'S'

C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\3715169118.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxSideP = df_test[df_test['deck'].isin(['A','B','C','E','G'])][df_test['side'].isna()].index
C:\Users\wuron\AppData\Local\Temp\ipykernel_20016\3715169118.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idxSideS = df_test[df_test['deck'].isin(['D','F','T'])][df_test['side'].isna()].index


In [46]:
# test

# df_test['isPort'] = [1 if i=='P' else 0 for i in df_test['side']]
# df_test['isVIP'] = [1 if i=='True' else 0 for i in df_test['VIP']]
# df_test['isCryoSleep'] = [1 if i=='True' else 0 for i in df_test['CryoSleep']]

df_test['isPort'] = [1 if i=='P' else 0 for i in df_test['side']]
df_test['isVIP'] = [1 if i=='True' else 0 for i in df_test['VIP']]
df_test['isCryoSleep'] = [1 if i=='True' else 0 for i in df_test['CryoSleep']]
df_test['fromEuropa'] = [1 if i=='Europa' else 0 for i in df_test['HomePlanet']]
df_test['to55'] = [1 if i=='55 Cancri e' else 0 for i in df_test['Destination']]
df_test['SPA200'] = [1 if i>=200 else 0 for i in df_test['Spa']]

In [47]:
# test

df_test_drop=df_test.drop(columns=['side', 'VIP', 'CryoSleep'])

# remove insignificant columns

# df_test_drop = df_test_drop.drop(columns=['FoodCourt', 'ShoppingMall', 'Spa'])

In [48]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(np.array(df_test_drop['Destination']).reshape(-1, 1))
df_test_drop[['Destination_0','Destination_1','Destination_2']] = enc.transform(np.array(df_test_drop['Destination']).reshape(-1, 1)).toarray()

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(np.array(df_test_drop['HomePlanet']).reshape(-1, 1))
df_test_drop[['HomePlanet_0','HomePlanet_1','HomePlanet_2']] = enc.transform(np.array(df_test_drop['HomePlanet']).reshape(-1, 1)).toarray()

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(np.array(df_test_drop['deck']).reshape(-1, 1))
df_test_drop[['deck_0','deck_1','deck_2','deck_3','deck_4','deck_5','deck_6','deck_7']] = enc.transform(np.array(df_test_drop['deck']).reshape(-1, 1)).toarray()

In [49]:
df_test_drop = df_test_drop.drop(columns=['Destination', 'HomePlanet', 'deck'])

In [50]:
test_pred = RFC.predict(df_test_drop)
rs = [True if i==1 else False for i in test_pred]
# rs = best_.predict(df_test_drop)

In [51]:
submission = pd.DataFrame()
submission['PassengerId'] = psgid
submission['Transported'] = rs
submission.to_csv('submission_valid.csv',index=False)

In [52]:
submission

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True
